In [ ]:
## In the command line, create a new environment
#conda create --name data_jamboree
## Activate the new environment
#conda activate data_jamboree
## Install packages
#conda install ipykernel jupyter ipython matplotlib matplotlib-inline 
#  notebook numpy  pandas  scikit-learn seaborn scipy statsmodels
#conda install -c conda-forge shap
#conda install -c plotly plotly
#pip install uszipcode geopy

In [ ]:
import warnings
warnings.simplefilter(action='ignore')
 
# pandas is a data frame package built 
# numpy is a numerical computing package
import pandas as pd  
import numpy as np  
from numpy.random import choice

# plotting libraries 
import matplotlib.pyplot as plt # python's "grandfather" 
import seaborn as sns # more modern and "pretty"
import plotly.express as px # interactive plotting

# packages for statistical tests 
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as mc
from scipy.stats import ttest_ind
from scipy.stats import f_oneway

# sklearn is python's most common "machine learning" package
# It is common to import the specific modules and classes you need
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
import sklearn.metrics as skm

# shap is a package for explaining machine learning models
import shap

# contains info from the US census
from uszipcode import SearchEngine

# geopy is a package for geocoding and reverse geocoding
from geopy.geocoders import GoogleV3, Nominatim


In [ ]:
## These lines of code show what versions that I used to run this notebook

# The ! tell jupyter to execute the command in the terminal (not python)
!python --version

In [ ]:
## Modules and versions

modules = ['numpy', 'pandas', 'matplotlib', 'seaborn', 'geopy', 'uszipcode', 'sklearn', 'shap']

for module in modules:
    try:
        mod = __import__(module)
        version = getattr(mod, '__version__', 'N/A')
        print(f"{module}: {version}")
    except ImportError:
        print(f"{module}: Not Installed")


In [ ]:
## Change pandas display options
## This will allow us to see more columns when we print a dataframe
pd.set_option('display.max_columns', 50)

# 0. Load and inspect the data

In [ ]:
#original_data = pd.read_csv('nyc311_011523-012123_by022023.csv') # read from local file

# GitHub URL 
url = 'https://github.com/statds/ids-s23/raw/8c68649925d069a1d93a71022b87b26a61b0c180/data/nyc311_011523-012123_by022023.csv'
original_data = pd.read_csv(url)

In [ ]:
df = original_data.copy()


In [ ]:
# look at shape, tail, and head
print(df.shape)
#df.head(2)
df.tail(2)

In [ ]:
# check the data types and missing values
df.info()

In [ ]:
#  I like to avoid doing math in my head
# get the number and pct of missing values in each column
df.isna().sum()

# fancy - This will show number and percent of missing values for each column
# pd.concat([df.isna().sum(), df.isna().mean()], axis=1).rename(columns={0: 'num', 1: 'pct'})


---

# 1. Data cleaning



### Task 1: Fix column names.

For ease of comparison across languages, make the column names consistent in style with lowercase using underscore to separate words within a name.



In [ ]:
## Two ways to rename columns

# Using rename and a lambda function
# df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))

# Using a list comprehension to reassign the columns
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [ ]:
print(df.columns)


### Task 2: Check for obvious errors or inefficiencies.



**Task 2a: For example, are there records whose Closed Date is earlier than or exactly the same as the Created Date?** 


In [ ]:
# convert all dates to datetime objects 
# pd.to_datetime() will raise an error by default if there are any invalid dates

df['created_date'] = pd.to_datetime(df['created_date'], format="%m/%d/%Y %I:%M:%S %p")
df['closed_date'] = pd.to_datetime(df['closed_date'], format="%m/%d/%Y %I:%M:%S %p")
df['due_date'] = pd.to_datetime(df['due_date'], format="%m/%d/%Y %I:%M:%S %p")
df['resolution_action_updated_date'] = pd.to_datetime(df['resolution_action_updated_date'], format="%m/%d/%Y %I:%M:%S %p")

In [ ]:
# Check for closed dates before or equal to created dates
errors = df['closed_date'] <= df['created_date']
print(errors.sum())

In [ ]:
# Create a column that indicates if the closed_date is missing (0), if closed date is before/same created date (-1) and 1 otherwise
# Missing closed dates are likely still censored (still open)
df['closed_date_indicator'] = np.where(df['closed_date'].isna(), 0, np.where(errors, -1, 1))
df['closed_date_indicator'].value_counts(dropna=False)

In [ ]:
df['closed_date'].isna().sum()


**Task 2b: Are there invalid values for any columns?** 

There are a lot of possible variable to consider.  I am going to look at
* zip code
* latitude/longitude
* borough

*1. Zip code*

In [ ]:
### Check that all zip codes are 5 digits
# Convert zip to a string and check the length of the zip codes
df['incident_zip'] = df['incident_zip'].astype('Int64').astype('str')
zip_length = df['incident_zip'].apply(len)
zip_length.value_counts(dropna=False)

In [ ]:
df.loc[zip_length==4,'incident_zip'].value_counts()

I found a dataset with all the zip codes in NYC [here](https://data.ny.gov/Government-Finance/New-York-State-ZIP-Codes-County-FIPS-Cross-Referen/juva-r6g2/data)

In [ ]:
### Check that all zip codes are in NYC

# read in zip code info (specify that all columns should be strings)
all_zips = pd.read_csv('New_York_State_ZIP_Codes-County_FIPS_Cross-Reference.csv', dtype=str)

# These are the counties in NYC
nyc_counties = ['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']

mapping = {'Bronx': 'Bronx',  'Kings': 'Brooklyn', 'New York': 'Manhattan', 
     'Queens': 'Queens', 'Richmond':'Staten Island'}

# df['Category'] = df['Category'].replace(mapping)
# print(df)

nyc_zips = all_zips.loc[all_zips['County Name'].isin(nyc_counties), ['ZIP Code', 'County Name']]
nyc_zips['Borough'] = nyc_zips['County Name'].replace(mapping)

# These are the zip codes in NYC
zips = nyc_zips['ZIP Code'].unique()
zips.sort() # this will change array in place
df[~df['incident_zip'].isin(zips)]['incident_zip'].value_counts(dropna=False)  



In [ ]:
## change <NA> to back to a missing value
df['incident_zip'].replace('<NA>', np.nan, inplace=True)
df['incident_zip'].isna().sum()

* 12345 is a zip code in Schenectady, NY (about 150 miles from NYC).  It could have been used as a placeholder.
* 10977 is a zip code in Sprint Valley, NY (close to NYC, so maybe valid?)
* 10000 does not appear to be a real zip code

In [ ]:
bad_zips = df[~df['incident_zip'].isin(zips)]['incident_zip'].unique()
bad_zips = bad_zips[1:]
bad_zips_df = df[df['incident_zip'].isin(bad_zips)][['incident_zip', 'latitude', 'longitude']]

In [ ]:
# # Later we'll restrict analysis to NYPD, so here I'm just looking at what bad zipcodes are with NYPD
# df[df['incident_zip'].isin(bad_zips)][['incident_zip', 'agency']]

To use the plotly function `scatter_mapbox()`, a free token from [mapbox](https://docs.mapbox.com/help/getting-started/access-tokens/) is required.  In this code, my mapbox token is in a file called `.mapbox_token`.  

In [ ]:
# If the mapbox token is not set, the map will not display
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(bad_zips_df, lon='longitude', lat='latitude', hover_data='incident_zip', zoom=9)
fig.update_layout(width=800, height=600)
fig.show()

Alternatively, the `scatter_geo` function also works, but I don't think that it looks as nice for this situation.

In [ ]:
# fig = px.scatter_geo(bad_zips_df, lon='longitude', lat='latitude', 
#                      hover_data='incident_zip',
#                       scope='usa')
# fig.update_layout(width=800, height=600)
# fig.update_geos(center=dict(lon=df['longitude'].mean(), lat=df['latitude'].mean()),
#                 projection_scale=100)
# fig.show()

*2. Latitude and Longitude*

(Check to see that all lat/lon values are inside the NYC limits)


In [ ]:
# Make sure the access token as been created
#px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(df, lon='longitude', lat='latitude', color='borough', hover_data='incident_zip',zoom=9)
fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
# fig = px.scatter_geo(df, lon='longitude', lat='latitude', color='borough',
#                      hover_data='incident_zip', scope='usa')
# fig.update_layout(width=800, height=600)
# fig.update_geos(center=dict(lon=df['longitude'].mean(), lat=df['latitude'].mean()),
#                 projection_scale=50)
# fig.show()

*3. Borough*

In [ ]:
# Use zip code to get unspecified boroughs

# Merge data with zip code/borough info
merged_df = df.merge(nyc_zips, left_on='incident_zip', right_on='ZIP Code', how='left')
# Fill 'Unspecified' borough values
merged_df.loc[merged_df['borough'] == 'Unspecified', 'borough'] = merged_df['Borough'].str.upper()
# rename back to "df" and drop the columns from the merge
df = merged_df.copy().drop(columns=['ZIP Code', 'County Name', 'Borough'])

In [ ]:
fig = px.scatter_mapbox(df, lon='longitude', lat='latitude', color='borough', hover_data='incident_zip',zoom=9)
fig.update_layout(width=800, height=600)
fig.show()

**Task 2c: Are any columns redundant?**

**Redundant Columns**

* Location and latitude/longitude contain redundant information
* X Coordinate and Y Coordinate "State Plane" is another time of coordinate system that is almost perfectly correlated with latitude and longitude for this small area
* There is a lot of overlapping information in `incident_address` and the location information in columns `street_name` thru `city`
* Agency and Agency name contain the same info
* Borough and Park Borough are identical



In [ ]:
df.columns

In [ ]:
# lat/lon correlation with x/y state plane coordinates
print(df['latitude'].corr(df['y_coordinate_(state_plane)']))
print(df['longitude'].corr(df['x_coordinate_(state_plane)']))

In [ ]:
pd.crosstab(index=df['borough'], columns=df['park_borough'],dropna=False)  

In [ ]:
## drop unnecessary columns if desired
df = df.drop(['agency_name','x_coordinate_(state_plane)', 'y_coordinate_(state_plane)',
              'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
              'intersection_street_1', 'intersection_street_2', 'address_type', 'city', 
              'location'], axis=1)

---
### Task 3: Fill in missing values if possible. 


**Task 3a: For example, if incident zip code is missing but the location is not, the zip code could be recovered by geocoding.**



In [ ]:
(df.incident_zip.isna() & ~df.longitude.isna()).sum()

In [ ]:
## Create variables for filling in missing zip codes
# df['zip_fill_nom'] = df['incident_zip']
# df['zip_fill_g'] = df['incident_zip']

# Geocode with Nominatim
#  This takes about 3 minutes to run

# def get_zipcode_from_nomin(row):
#     if pd.isnull(row['zip_fill_nom']) and not pd.isnull(row['latitude']) and not pd.isnull(row['longitude']):
#         geolocator = Nominatim(user_agent="data_jamboree")
#         location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
#         address = location.raw.get("address", {})
#         return address.get("postcode", None)
           
#     return row['zip_fill_nom']


# df.loc[pd.isnull(df['zip_fill_nom']), 'zip_fill_nom'] = df[pd.isnull(df['zip_fill_nom'])].apply(get_zipcode_from_nomin, axis=1)


In [ ]:
# This takes about 2 minutes to run

# with open('apikey.txt') as f:
#     apikey = f.readline().strip()

# def get_zipcode_from_google(row):
#     if pd.isnull(row['zip_fill_g']) and not pd.isnull(row['latitude']) and not pd.isnull(row['longitude']):
#         geolocator = GoogleV3(api_key=apikey)
#         location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
#         data = location.raw['address_components']
#         postal_code = next((item['short_name'] for item in data if 'postal_code' in item['types']), None)
#         return postal_code
#     return row['zip_fill_g']  # This line returns the existing value if conditions aren't met.

# # Use loc to target rows with missing 'incident_zip' values
# df.loc[pd.isnull(df['zip_fill_g']), 'zip_fill_g'] = df[pd.isnull(df['zip_fill_g'])].apply(get_zipcode_from_google, axis=1)


In [ ]:
# For some reason, sometimes one method resulted in a "None" value while the
# other method came back with a valid zipcode.  I am going to combine the 
# columns so that the missing values of the Google method will be filled with the 
# missing values of the other method

#df['incident_zip'] = df['zip_fill_g'].combine_first(df['zip_fill_nom'])
#df.drop(['zip_fill_g', 'zip_fill_nom'], axis=1, inplace=True)
#df.to_csv('nyc311_with_geo.csv', index=False)

In [ ]:
df = pd.read_csv('nyc311_with_geo.csv', 
                 parse_dates=['created_date', 'closed_date', 'due_date',
                             'resolution_action_updated_date'],
                 dtype={'incident_zip': 'str'})


---
### Summarize your suggestions to the data curator in several bullet points.

---


# 2. Data manipulation. 
**Focus only on requests made to NYPD.**


In [ ]:
nypd = df[df['agency'] == 'NYPD'].copy()
nypd.dropna(subset=['incident_zip'], inplace=True)

In [ ]:
nypd['closed_date_indicator'].value_counts(dropna=False)

In [ ]:
#nypd.info()


### Task 1: Create duration variable.

**Create a a new variable duration, which represents the time period from the Created Date to Closed Date. Note that duration may be censored for some requests.**


In [ ]:
# set missing closed dates to the maxiumum closed date
nypd.loc[nypd['closed_date'].isna(), 'closed_date'] = nypd['closed_date'].max()


In [ ]:
# duration in hours
nypd['duration'] = (nypd['closed_date'] - nypd['created_date']).dt.total_seconds() / 3600


### Task 2: Visualize duration

**Visualize the distribution of uncensored duration by weekdays/weekend and by borough, and test whether the distributions are the same across weekdays/weekends of their creation and across boroughs.**


In [ ]:
nypd['created_daytype'] = nypd['created_date'].dt.dayofweek.map(lambda x: 'weekend' if x >= 5 else 'weekday')
nypd['created_hour'] = nypd['created_date'].dt.hour
uncensored = nypd['closed_date_indicator'] == 1  

#### Weekend vs Weekday

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,8))
sns.boxplot(x='created_daytype', y='duration', data=nypd[uncensored], ax=ax[0])
ax[0].set_title('Distribution of Duration by Day Type')

sns.boxplot(x='created_daytype', y='duration', showfliers=False,   data=nypd[uncensored], ax=ax[1])
ax[1].set_title('Distribution of Duration by Day Type (without outliers)')
plt.show()

In [ ]:
nypd.groupby('created_daytype')['duration'].describe()

In [ ]:
# Testing durations across weekdays/weekends
weekday_data = nypd[(nypd['created_daytype'] == 'weekday') & uncensored]['duration']
weekend_data = nypd[(nypd['created_daytype'] == 'weekend') & uncensored]['duration']

stat, p = ttest_ind(weekday_data, weekend_data)
print(f"Difference in mean: {np.mean(weekday_data) - np.mean(weekend_data)}")
print(f"p-value for weekday vs weekend duration: {p}")

In [ ]:
# Define a function to compute the difference in means
def mean_diff(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

# Bootstrap confidence intervals
num_bootstraps = 10000
alpha = 0.05
lower_percentile = 100 * alpha/2
upper_percentile = 100 * (1 - alpha/2)

# Store the differences
bootstrap_diffs = []

for _ in range(num_bootstraps):
    # Resample each group
    boot1 = choice(weekday_data, size=len(weekday_data), replace=True)
    boot2 = choice(weekend_data, size=len(weekend_data), replace=True)
    
    # Compute the difference in medians
    diff = mean_diff(boot1, boot2)
    bootstrap_diffs.append(diff)

# Compute the percentiles for the confidence intervals
ci_lower = np.percentile(bootstrap_diffs, lower_percentile)
ci_upper = np.percentile(bootstrap_diffs, upper_percentile)

print(f"95% CI for the difference in medians between group1 and group2: [{ci_lower}, {ci_upper}]")


#### Borough

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,8))
sns.boxplot(x='borough', y='duration', data=nypd[uncensored], ax=ax[0])
ax[0].set_title('Distribution of Duration by Borough')

sns.boxplot(x='borough', y='duration', showfliers=False,   data=nypd[uncensored], ax=ax[1])
ax[1].set_title('Distribution of Duration by Borough (without outliers)')
plt.show()

In [ ]:
nypd.groupby('borough')['duration'].describe()

In [ ]:
# Testing durations across boroughs
boroughs = nypd['borough'].dropna().unique()
borough_data = [nypd[(nypd['borough'] == borough) & uncensored]['duration'] for borough in boroughs if pd.notnull(borough)]

stat, p = f_oneway(*borough_data)
print(f"p-value for durations across boroughs: {p}")

In [ ]:
# Combine the groups into a single data array and a corresponding array of group labels
data = nypd.dropna(subset=['duration','borough']).loc[uncensored,'duration']
groups = nypd.dropna(subset=['duration','borough']).loc[uncensored,'borough']

# Perform the Tukey HSD test
tukey_result = mc.pairwise_tukeyhsd(data, groups, alpha=0.05)
print(tukey_result)



### Task 3: Merge with zipcode census data

**Basic information at the zipcode level such as population density, median home value, and median household income is available from the US Census. Convenient accesses are, for example, R package zipcodeR and Python package uszipcode; there seems to no Julia equivalent yet but Julia can call R or Python easily. Merge the zipcode level information with the NYPD requests data.**



In [ ]:
## Drop the rows with wrong zip codes
nypd = nypd[nypd!='10000']

In [ ]:
#nypd.incident_zip.unique().sort()

In [ ]:
## census data by zip
search = SearchEngine()
unique_zips = nypd['incident_zip'].dropna().unique().astype('int').astype('str')
zip_data = [search.by_zipcode(zipcode) for zipcode in unique_zips]
zip_data = pd.DataFrame([z.to_dict() for z in zip_data])



In [ ]:
zip_data.head()

In [ ]:
merged_df = pd.merge(nypd, zip_data, how='left', left_on='incident_zip', right_on='zipcode')

In [ ]:
# Fill in missing lat/lon with values from zipcode dataset
merged_df['latitude'] = merged_df['latitude'].fillna(merged_df['lat'])
merged_df['longitude'] = merged_df['longitude'].fillna(merged_df['lng'])

In [ ]:
# drop columns that have all missing values
merged_df.dropna(axis=1, how='all', inplace=True)

---


# 3. Data analysis.


### Task 1: Create the target variable
**Define a binary variable over3h which is 1 if duration is greater than 3 hours. Note that it can be obtained even for censored duration.**


In [ ]:
merged_df['target'] = np.where(merged_df['duration'] > 3, 1, 0)

In [ ]:
#nypd['target'].value_counts()
merged_df['target'].value_counts(normalize=True)

In [ ]:
# exclude complaint_type for now
model_df = merged_df[['complaint_type', 'borough', 'latitude', 'longitude', 
                     'created_daytype', 'created_hour', 'population', 'population_density', 
                     'housing_units', 'water_area_in_sqmi', 'occupied_housing_units', 
                     'median_home_value', 'median_household_income', 'target']].copy()

In [ ]:
#### Keep only top 5 categories and combine others into 'other'

category_counts = model_df['complaint_type'].value_counts()

# Get the 10 least common categories

n_combine = len(category_counts) - 5
least_common_categories = category_counts.nsmallest(n_combine).index

# Replace these categories with 'other'
model_df['complaint_type'] = model_df['complaint_type'].replace(least_common_categories, 'other')

model_df['complaint_type'].value_counts()

In [ ]:
### Few missing values and many are in the same rows 
## I am going to drop the rows with missing values
model_df.dropna(inplace=True)

In [ ]:
num_cols = model_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_cols.remove('target')
cat_cols = model_df.select_dtypes(include=['object']).columns.tolist()

In [ ]:
# Scale numeric columns
scaler = StandardScaler()
model_df[num_cols] = scaler.fit_transform(model_df[num_cols])

# Convert categorical columns to dummies
model_df = pd.get_dummies(model_df, columns=cat_cols, drop_first=True)

In [ ]:
# Some of the new columns have spaces and special characters that will cause problems later
model_df.columns = [col.lower().replace(' ', '_') for col in model_df.columns]
model_df.columns = [col.replace('-', '') for col in model_df.columns]
model_df.columns = [col.replace('/', '') for col in model_df.columns]


In [ ]:
X = model_df.drop(['target'], axis=1)
y = model_df['target']

### Task 2: Build a logistic model 
**Build a logistic model to predict over3h using the 311 request data as well as those zip code level covariates. If your model has tuning parameters, justify their choices. Use appropriate metrics to assess the performance of the model.**



---
### Using `statsmodels`

In [ ]:
formula = 'target ~ ' + '+'.join(X.columns)
print(formula)

In [ ]:
model = smf.logit(formula=formula, data=model_df)
result = model.fit()
probabilities = result.predict(model_df)
predictions = (probabilities > 0.5).astype(int)

print(result.summary())

In [ ]:
skm.accuracy_score(y, predictions)

In [ ]:
skm.confusion_matrix(y, predictions)

### Using `sklearn`

In [ ]:
#lr = LogisticRegression(solver='liblinear', scoring='recall', Cs=20)
#lr.fit(X,y)
lr = LogisticRegression(solver='liblinear', C=.0001)

In [ ]:
## Since I am not using a trainig and test set, I am going to use cross validation
cross_val_score(lr, X, y, cv=10).mean()
#cross_val_score(lr, X, y, scoring='f1', cv=10).mean()

In [ ]:
cv_scores = cross_validate(lr, X, y, scoring=['accuracy', 'precision', 'recall', 'f1'], cv=10)
for key in cv_scores.keys():
    print(f"{key}: {cv_scores[key].mean():.3f} +/- {cv_scores[key].std():.3f}")

In [ ]:
skm.confusion_matrix(y, predictions)

In [ ]:
cv_predictions = cross_val_predict(lr, X, y, cv=10)
cv_probabilities = cross_val_predict(lr, X, y, cv=10, method='predict_proba')[:,1]

In [ ]:
# Compute ROC curve and AUC
fpr, tpr, thresholds = skm.roc_curve(y, probabilities)
roc_auc = skm.auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
skm.confusion_matrix(y, cv_predictions)

In [ ]:
print(skm.classification_report(y, cv_predictions))

In [ ]:
### These are different from statsmodels because scikit-learn uses regularization by default!
lr.fit(X,y)
print(lr.intercept_)
print(lr.coef_)

### Task 3: Repeat the analysis with another model (e.g., random forest; neural network; etc.).

In [ ]:
rf = RandomForestClassifier()
cv_scores_rf = cross_validate(rf, X, y, scoring=['accuracy', 'precision', 'recall', 'f1'], cv=10)
for key in cv_scores_rf.keys():
    print(f"{key}: {cv_scores_rf[key].mean():.3f} +/- {cv_scores_rf[key].std():.3f}")

In [ ]:
pred_rf = cross_val_predict(rf, X, y)
prob_rf = cross_val_predict(rf, X, y, method='predict_proba')[:,1]

In [ ]:
skm.confusion_matrix(y, pred_rf)

In [ ]:
print(skm.classification_report(y, pred_rf))

In [ ]:
# Compute ROC curve and AUC
fpr, tpr, thresholds = skm.roc_curve(y, prob_rf)
roc_auc = skm.auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
rf.fit(X,y)

In [ ]:
pd.Series(data=rf.feature_importances_, index=X.columns).sort_values(ascending=False)

### SHAP

- **Intuitive Model Interpretations:** SHAP (SHapley Additive exPlanations) provides a unified measure of feature importance for any machine learning model, based on game theory
  
- **From Global to Local:** Not only does SHAP help in understanding the overall impact of features across the model (global interpretability), but it also dissects individual predictions to show the contribution of each feature (local interpretability)

- **Versatility and Adoption:** SHAP is model-agnostic, meaning it can be applied to any machine learning model, from simple linear regressions to complex deep neural networks

In [ ]:
features = X.columns

In [ ]:
## Since the dataset is so large, this took a long time to run (about 3 hours)
## Using a testing dataset and/or few features will speed up the process

# explainer = shap.TreeExplainer(rf)
# explainer.feature_names = features
# sv = explainer(X)
# shap_values = shap.Explanation(sv[:,:,1], sv.base_values[:,1], X, feature_names=features)

# # I saved the results to a pickle file so I don't have to run it again

# import pickle
# with open('shap_values.pkl', 'wb') as file:
#     pickle.dump((shap_values, X), file)


In [ ]:
# To load it back:
import pickle
with open('shap_values.pkl', 'rb') as file:
    shap_values, X = pickle.load(file)

In [ ]:
shap.plots.bar(shap_values, max_display=25)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=25)

In [ ]:
shap.plots.waterfall(shap_values[0])